## Machine Learning Examples
Jeff Lockhart

UM Data Camp

Tuesday, 18 June, 2019

Slides: https://docs.google.com/presentation/d/1QT3DzB1sampdsXCMWbU4q1pnwdGL18R52O5G2ELwmgE/edit?usp=sharing


In [ ]:
import pandas as pd
import numpy as np
from sklearn import cluster, datasets, metrics, preprocessing
from sklearn import linear_model, svm, neighbors, gaussian_process, tree, neural_network, ensemble, naive_bayes
from sklearn.model_selection import train_test_split, cross_val_predict
import matplotlib.pyplot as plt

%matplotlib inline

## Basic clustering example

In [ ]:
#make some data
n_samples = 2000
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
X = pd.DataFrame(X)
X.columns = ['x0', 'x1']
X.head()

In [ ]:
#plot it
plt.figure(figsize=(10, 10))
plt.scatter(X['x0'], X['x1'])

In [ ]:
#cluster with kmeans
model = cluster.KMeans(n_clusters=3, random_state=random_state)
y_pred = model.fit_predict(X)
y_pred

In [ ]:
#plot it
plt.figure(figsize=(10, 10))
plt.scatter(X['x0'], X['x1'], c=y_pred)

### Evaluate quality
There are many measures of cluster quality. Here I just demonstrate one of them.

In [ ]:
metrics.silhouette_score(X, y_pred, metric='euclidean')

## Clustering comparison
Example from: https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html

In [ ]:
import time
import warnings

from sklearn import mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

In [ ]:
# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)


In [ ]:
# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()

## Prediction
### Regression basic example


In [ ]:
chicago = pd.read_csv('community_area_stats.tsv', sep='\t')
chicago.head()

In [ ]:
chicago = chicago[chicago.community_area_number>0]

In [ ]:
chicago.columns

In [ ]:
y = chicago['2010_life_expectancy']
x = chicago[['percent_of_housing_crowded', 'percent_households_below_poverty',
             'percent_aged_16+_unemployed', 'percent_aged_25+_without_high_school_diploma', 
             'per_capita_income_', 'hardship_index', 'low_birth_weight',
             'assault_(homicide)', 'cancer_(all_sites)', 'firearm-related',
             'infant_mortality_rate', #'childhood_lead_poisoning', 
             'police_complaints_per_thousand', 'total_population', 'median_age', 
             'pct_white', 'pct_vacant_housing', 'pct_rental_housing', 
             'total_housing_units', #'pct_affordable_housing'
            ]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.shape

In [ ]:
x_test.shape

In [ ]:
model = linear_model.LinearRegression()
model.fit(x_train, y_train)

In [ ]:
coefficients = pd.DataFrame(zip(x.columns, model.coef_))
coefficients.columns = ['Variable', 'Coefficient']
coefficients.sort_values(by='Coefficient', ascending=False, inplace=True)
coefficients.round(3)

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
metrics.mean_squared_error(y_test, y_pred)

In [ ]:
metrics.r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(y_test, y_pred, color='black')
plt.plot([65,88], [65,88], label='y=x')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Life Expectancy')
plt.legend()

In [ ]:
y_pred = cross_val_predict(model, x, y, cv=10)
y_pred

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(y, y_pred, color='black')
plt.plot([65,88], [65,88], label='y=x')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Life Expectancy')
plt.legend()

In [ ]:
metrics.r2_score(y, y_pred)

In [ ]:
metrics.mean_squared_error(y, y_pred)

## Comparing regression models 

In [ ]:
models = {'OLS': linear_model.LinearRegression(),
          'Lasso': linear_model.Lasso(alpha=.1),
          'Ridge': linear_model.Ridge(alpha=.5),
          'Passive_Aggressive': linear_model.PassiveAggressiveRegressor(max_iter=100),
          'SVM': svm.SVR(gamma='auto'),
          #'Stochastic_Gradient_Descent': linear_model.SGDRegressor(),
          '3_Nearest_Neighbor': neighbors.KNeighborsRegressor(n_neighbors=3),
          '5_Nearest_Neighbor': neighbors.KNeighborsRegressor(n_neighbors=5),
          'Gausian_Process': gaussian_process.GaussianProcessRegressor(),
          'Decision_Tree': tree.DecisionTreeRegressor(),
          'Random_Forest': ensemble.RandomForestRegressor(n_estimators=20),
          'Ada_Boost': ensemble.AdaBoostRegressor(),
          'Neural_Network': neural_network.MLPRegressor(hidden_layer_sizes=(20,10,5), solver='lbfgs', max_iter=100)
         }

results = []

for name, model in models.items():
    print('Trying', name)
    tmp = dict()
    tmp['model'] = name
    y_pred = cross_val_predict(model, x, y, cv=10)
    tmp['MSE'] = metrics.mean_squared_error(y, y_pred)
    tmp['r2'] = metrics.r2_score(y, y_pred)
    results.append(tmp)

results = pd.DataFrame(results)
results = results[['model', 'MSE', 'r2']]
results.sort_values(by='MSE').round(2)

## Classification

In [ ]:
gss = pd.read_csv('GSS2012Extract_PresApproval_StrongPartyAffiliation_CLEAN.csv')
gss.shape

In [ ]:
gss.head()

In [ ]:
gss['approve'] = (gss.pres_approval == 'Approve')
gss.head()

In [ ]:
gss['approve'] = (gss.pres_approval == 'Approve').astype(int)
gss.head()

In [ ]:
gss['democrat'] = (gss.party.str.contains('democrat')).astype(int)
gss.head()

In [ ]:
gss = pd.get_dummies(gss, drop_first=True, prefix='region', columns=['region_of_interview'])
gss.head()

In [ ]:
gss.columns

In [ ]:
x = gss[['age', 'size', 'democrat', 'region_E. sou. central', 
         'region_Middle atlantic', 'region_Mountain', 'region_New england', 
         'region_Pacific', 'region_South atlantic', 'region_W. nor. central',
         'region_W. sou. central']]
y = gss.approve

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
model = linear_model.LogisticRegression(solver='liblinear')
model.fit(x_train, y_train)

In [ ]:
coefficients = pd.DataFrame(zip(x.columns, model.coef_[0]))
coefficients.columns = ['Variable', 'Coefficient']
coefficients.sort_values(by='Coefficient', ascending=False, inplace=True)
coefficients.round(3)

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
print('accuracy:', metrics.accuracy_score(y_test, y_pred))
print('precision:', metrics.precision_score(y_test, y_pred))
print('recall:', metrics.recall_score(y_test, y_pred))
print('f1:', metrics.f1_score(y_test, y_pred))

### Advanced evaluation

In [ ]:
y_pred = model.predict_proba(x_test)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['pr(0)', 'pr(1)']
y_pred.head()

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred['pr(1)'])
auc = metrics.roc_auc_score(y_test, y_pred['pr(1)'])

In [ ]:
plt.figure(figsize=(8,8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend()
plt.show()

In [ ]:
models = {'Logit': linear_model.LogisticRegression(solver='liblinear'),
          'SVM': svm.SVC(gamma='auto', probability=True),
          'Stochastic_Gradient_Descent': linear_model.SGDClassifier(),
          '3_Nearest_Neighbor': neighbors.KNeighborsClassifier(n_neighbors=3),
          'Gausian_Process': gaussian_process.GaussianProcessClassifier(),
          'Decision_Tree': tree.DecisionTreeClassifier(),
          'Random_Forest': ensemble.RandomForestClassifier(n_estimators=20),
          'Ada_Boost': ensemble.AdaBoostClassifier(),
          'Neural_Network': neural_network.MLPClassifier(hidden_layer_sizes=(20,10,5), solver='lbfgs', max_iter=100),
          'Naive_Bayes':GaussianNB()
         }

results = []

for name, model in models.items():
    print('Trying', name)
    tmp = dict()
    tmp['model'] = name
    y_pred = cross_val_predict(model, x, y, cv=10) # method='predict_proba'
    #tmp['AUC'] = metrics.roc_auc_score(y, y_pred)
    tmp['Accuracy'] = metrics.accuracy_score(y, y_pred)
    tmp['Precision'] = metrics.precision_score(y, y_pred)
    tmp['Recall'] = metrics.recall_score(y, y_pred)
    results.append(tmp)

results = pd.DataFrame(results)
results.sort_values(by='Accuracy', ascending=False).round(2)

In [ ]:
results = []

for name, model in models.items():
    print('Trying', name)
    tmp = dict()
    try:
        tmp['model'] = name
        y_pred = cross_val_predict(model, x, y, cv=10, method='predict_proba')
        tmp['AUC'] = metrics.roc_auc_score(y, y_pred[:,1:2])
        fpr, tpr, thresholds = metrics.roc_curve(y, y_pred[:,1:2])
        tmp['fpr'] = fpr
        tmp['tpr'] = tpr
        results.append(tmp)
    except:
        pass

plt.figure(figsize=(8,8))
lw = 2
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
for r in results:
    plt.plot(r['fpr'], r['tpr'], lw=lw, label=r['model']+' (AUC = %0.2f)' % r['AUC'])
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend()
plt.show()

## Classifier comparison
example from: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

figure = plt.figure(figsize=(27, 9))
i = 1
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    # Plot the testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # Plot the testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.show()